In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough




In [3]:
llm = ChatOpenAI(model="gpt-3.5-turbo")



In [52]:
loader = PyPDFLoader("files/DPMG.pdf")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    # chunk_size=1000, chunk_overlap=0,
    # separators=["."]
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):

    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [53]:
rag_chain.invoke("Quais as datas do Lucas de Sousa Assunção?")

'Lucas de Sousa Assunção teve aulas nos dias 18 de abril e 23 de abril de 2024, ambos no turno noturno. Ele estava presente junto com outros alunos como Matheus Marinho Feitoza, Paulo Henrique dos Santos, e outros mencionados nas listas.'

In [39]:
retriever.invoke("Quais os plantões do Lucas de Sousa Assunção?")

[Document(metadata={'page': 4, 'source': 'files/DPMG.pdf'}, page_content='Lucas De Sousa Assunção;\nMatheus Marinho Feitoza;\nPaulo Henrique dos Santos;\nLuísa Ribeiro Cassini;\nIsabella Torres Godoi;\nHugo Teodoro Calandrini De Azevedo Melo;\nVictor Henrique Batista de Macedo; e\nAntônio Lopes Júnior.\n23 de abril de 2024 NoturnoLuiz Henrique Costa Gonçalves;\nCristiano Moreira Moteram;\nMatheus Marinho Feitoza;\nTaylan Silva Bustamante;\nPaulo Henrique dos Santos ;\nHugo Teodoro Calandrini De Azevedo Melo;\nEduardo Thierry Batista Leite;\nAntônio Lopes Júnior; e\nFlávio Júlio Alves Silva.\nATO Nº 6302/2024\nO DEFENSOR PÚBLICO-GERAL DO ESTADO DE MINAS  GERAIS em exercício, na forma do artigo 12, §1°,  da Lei\nComplementar n. 65/03, no uso de atribuição estabelecida no artigo 9º, XXXVIII, da Lei Complementar Estadual n.\n65, de 16 de janeiro de 2003, considerando o que consta no procedimento SEI n. 9990000001.003547/2024-29;\nautoriza o Defensor Público Flávio Rodrigues Lélles, Madep n

In [68]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Você é um assistente para tarefas de resposta a perguntas. "
    "Use os seguintes pedaços de contexto recuperado para responder "
    "à pergunta. Se você não sabe a resposta, diga que "
    "não sabe. Use no máximo três frases e mantenha a "
    "resposta concisa."
    "\n\n"
    "{context}"
    "Pergunta: Datas do Lucas"
    "Resposta: 10/04/22"
    
    "Pergunta: Datas do Antonio"
    "Resposta: 10/04/22 e 10/05/22"
          
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [69]:
rag_chain.invoke({"input":"Datas do Lucas De Sousa Assunção"})

{'input': 'Datas do Lucas De Sousa Assunção',
 'context': [Document(metadata={'page': 4, 'source': 'files/DPMG.pdf'}, page_content='Lucas De Sousa Assunção;\nMatheus Marinho Feitoza;\nPaulo Henrique dos Santos;\nLuísa Ribeiro Cassini;\nIsabella Torres Godoi;\nHugo Teodoro Calandrini De Azevedo Melo;\nVictor Henrique Batista de Macedo; e\nAntônio Lopes Júnior.\n23 de abril de 2024 NoturnoLuiz Henrique Costa Gonçalves;\nCristiano Moreira Moteram;\nMatheus Marinho Feitoza;\nTaylan Silva Bustamante;\nPaulo Henrique dos Santos ;\nHugo Teodoro Calandrini De Azevedo Melo;\nEduardo Thierry Batista Leite;\nAntônio Lopes Júnior; e\nFlávio Júlio Alves Silva.\nATO Nº 6302/2024\nO DEFENSOR PÚBLICO-GERAL DO ESTADO DE MINAS  GERAIS em exercício, na forma do artigo 12, §1°,  da Lei\nComplementar n. 65/03, no uso de atribuição estabelecida no artigo 9º, XXXVIII, da Lei Complementar Estadual n.\n65, de 16 de janeiro de 2003, considerando o que consta no procedimento SEI n. 9990000001.003547/2024-29;\nau

In [31]:
result["answer"]

'Não há informações disponíveis sobre os plantões específicos de Lucas De Sousa Assunção nos documentos fornecidos.'

In [26]:
res = retriever.invoke("Quais são os plantões do Lucas de Sousa Assunção")

In [28]:
for row in res:
    print(row)

page_content='Lucas De Sousa Assunção;
Matheus Marinho Feitoza;
Paulo Henrique dos Santos;
Luísa Ribeiro Cassini;
Isabella Torres Godoi;
Hugo Teodoro Calandrini De Azevedo Melo;
Victor Henrique Batista de Macedo; e
Antônio Lopes Júnior.
23 de abril de 2024 NoturnoLuiz Henrique Costa Gonçalves;
Cristiano Moreira Moteram;
Matheus Marinho Feitoza;
Taylan Silva Bustamante;
Paulo Henrique dos Santos ;
Hugo Teodoro Calandrini De Azevedo Melo;
Eduardo Thierry Batista Leite;
Antônio Lopes Júnior; e
Flávio Júlio Alves Silva.
ATO Nº 6302/2024
O DEFENSOR PÚBLICO-GERAL DO ESTADO DE MINAS  GERAIS em exercício, na forma do artigo 12, §1°,  da Lei
Complementar n. 65/03, no uso de atribuição estabelecida no artigo 9º, XXXVIII, da Lei Complementar Estadual n.
65, de 16 de janeiro de 2003, considerando o que consta no procedimento SEI n. 9990000001.003547/2024-29;
autoriza o Defensor Público Flávio Rodrigues Lélles, Madep n. 419, a se ausentar das suas atribuições ordinárias e' metadata={'page': 4, 'sou

In [27]:
for row in res["context"]:
    print(row, end="##### \n\n")
    
    

TypeError: list indices must be integers or slices, not str

In [1]:
import pdfminer

with pdfminer.PDFPage('files/DPMG.pdf') as page:
    texto = page.extract_text()
    print(texto)

AttributeError: module 'pdfminer' has no attribute 'PDFPage'